In [1]:
from google.colab import drive
drive.mount('/content/drive')#allocation my drive space to this note book, so i can use the path
#packages that are useful
import numpy as np #linear algebra
import pandas as pd #data processing, CSV file I/O (e.g. pd.read_csv)
import os #functions for interacting with the operating system
import pandas as pd #data analysis toolkit
import numpy as np #for array processing
import matplotlib.pyplot as plt #plot the figures
%matplotlib inline 
import seaborn as sns#data visualization
import re #regularization
import nltk #dealing with nature language
from nltk.corpus import  stopwords #massive dump of all kinds of natural language data sets
from nltk.stem.porter import PorterStemmer #Stem extraction based on porter
from nltk.tokenize import word_tokenize, sent_tokenize #using for text spliting
from nltk.stem.wordnet import WordNetLemmatizer #Same word form induction, used for dimensionality reduction
from sklearn.model_selection import train_test_split #split the dataset into training set and test set
from sklearn.model_selection import cross_val_score, GridSearchCV #Greedy algorithm automatically adjusts parameters
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer #Text preprocessing based on word frequency
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import string #string methods
import random 
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

Mounted at /content/drive
/content/drive/MyDrive/Final_project/dataset_amazon/Musical_instruments_reviews.csv
/content/drive/MyDrive/Final_project/dataset_amazon/Musical_Instruments_5.json
/content/drive/MyDrive/Final_project/dataset_amazon/fuzzydataset.csv
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
data_raw = pd.read_csv('/content/drive/MyDrive/Final_project/dataset_amazon/fuzzydataset.csv')
data_raw.head()
data_raw.columns = data_raw.columns.str.lower() #Change the name of each column to lowercase 
print(data_raw.columns) #Preview each column name
print(data_raw.isnull().sum()) #Confirm that there is no blank data in reviewtext and summary.
print('Now the shape of our data is {} x {}.'.format(data_raw.shape[0], data_raw.shape[1])) #Preview the dimension of data

Index(['reviewerid', 'asin', 'reviewername', 'helpful', 'reviewtext',
       'overall', 'summary', 'unixreviewtime', 'reviewtime'],
      dtype='object')
reviewerid        0
asin              0
reviewername      1
helpful           0
reviewtext        0
overall           0
summary           0
unixreviewtime    0
reviewtime        0
dtype: int64
Now the shape of our data is 300 x 9.


In [3]:
#use sentiment classes to describe the overall values
data_raw['sentiment'] = data_raw.overall.replace({
    1:'negative',
    2:'negative',
    3:'neutral',
    4:'positive',
    5:'positive'
})

In [4]:
#create the nature language data with summary + review, and the sentiment is the output
X_raw = data_raw['reviewtext'] + ' ' + data_raw['summary']
y_test = data_raw['sentiment']
#convert the  raw language data into string
X_raw = X_raw.astype(str)

In [5]:
#Preprocessing of language data
def nlpre(X_raw):
    
    ##sub functions
    #punctuations remove function
    def remove_punct(X_fcn):
        string1 = X_fcn.lower()
        translation_table = dict.fromkeys(map(ord, string.punctuation),' ') 
        string2 = string1.translate(translation_table)
        return string2
    #stopwords remove function  
    def remove_stopwords(X_fcn):
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*') 
        string2 = pattern.sub(' ', X_fcn)
        return string2
    #tokenizing function 
    def tokenize_words(X_fcn):
        words = nltk.word_tokenize(X_fcn) 
        return words
    #lemmatizing function 
    lemmatizer = WordNetLemmatizer()
    def lemmatize_words(X_fcn):
        words = lemmatizer.lemmatize(X_fcn) 
        return words

    ##Processing part
    X_raw_clear_punct = []
    for i in range(len(X_raw)):
        test_data = remove_punct(X_raw[i]) #replace !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ with space
        X_raw_clear_punct.append(test_data)
        
    X_raw_clear_stopwords = []
    for i in range(len(X_raw)):
        test_data = remove_stopwords(X_raw[i]) #remove the words that does not have so much meaning in this process
        X_raw_clear_stopwords.append(test_data)
        
    X_raw_tokenized_words = []
    for i in range(len(X_raw)):
        test_data = tokenize_words(X_raw[i]) #devide the string into substring
        X_raw_tokenized_words.append(test_data)
   
    X_raw_lemmatized_words = []
    for i in range(len(X_raw)):
        test_data = lemmatize_words(X_raw[i]) #merge the same words
        X_raw_lemmatized_words.append(test_data)
        
    #creating the bag of words model
    cv = CountVectorizer(max_features=1000) #Declare a word to word vector transformation，specifies the number of elements of the vectorized corpus
    X_raw_vector = cv.fit_transform(X_raw_lemmatized_words).toarray() #convert to matrix
    
    tfidf = TfidfTransformer()
    X_processed_tfidf = tfidf.fit_transform(X_raw_vector).toarray() #scale down the impact of tokens that occur very frequently in a given corpus
    
    return X_processed_tfidf
    #use nlp preprocessing and split dataset into training set and test set
X_test = nlpre(X_raw)

In [6]:
# convert sentiments state in y_test to dummies
y_test_dummies = pd.get_dummies(y_test).values
print('Shape of Label tensor: ', y_test_dummies.shape)

Shape of Label tensor:  (300, 3)


In [8]:
# model test
from keras.models import load_model

model = load_model('/content/drive/MyDrive/Final_project/model/Amazon_Instru_Sentiment_Analy.h5')
scores = model.evaluate(X_test, y_test_dummies)

LSTM_accuracy = scores[1]*100

print('test accuracy: ', LSTM_accuracy, '%')

10/10 [==============================] - 2s 204ms/step - loss: 0.3057 - accuracy: 0.9300
test accuracy:  93.00000071525574 %
